In [3]:
!cd github & git clone https://github.com/Farama-Foundation/ViZDoom.git

Cloning into 'ViZDoom'...
Updating files:  33% (686/2058)
Updating files:  34% (700/2058)
Updating files:  35% (721/2058)
Updating files:  36% (741/2058)
Updating files:  37% (762/2058)
Updating files:  38% (783/2058)
Updating files:  39% (803/2058)
Updating files:  40% (824/2058)
Updating files:  41% (844/2058)
Updating files:  42% (865/2058)
Updating files:  43% (885/2058)
Updating files:  44% (906/2058)
Updating files:  45% (927/2058)
Updating files:  46% (947/2058)
Updating files:  47% (968/2058)
Updating files:  48% (988/2058)
Updating files:  49% (1009/2058)
Updating files:  50% (1029/2058)
Updating files:  51% (1050/2058)
Updating files:  52% (1071/2058)
Updating files:  53% (1091/2058)
Updating files:  54% (1112/2058)
Updating files:  55% (1132/2058)
Updating files:  56% (1153/2058)
Updating files:  57% (1174/2058)
Updating files:  58% (1194/2058)
Updating files:  59% (1215/2058)
Updating files:  60% (1235/2058)
Updating files:  61% (1256/2058)
Updating files:  62% (1276/2058)


In [11]:
!pip install gym

  Using cached gym-0.26.2.tar.gz (721 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached gym_notices-0.0.8-py3-none-any.whl.metadata (1.0 kB)
Using cached gym_notices-0.0.8-py3-none-any.whl (3.0 kB)
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827635 sha256=5fe3e4c6d75241f9a05799fc010d63f54b2a89873633975c05dccf63574d52d8
  Stored in directory: c:\users\19096\appdata\local\pip\cache\wheels\95\51\6c\9bb05ebbe7c5cb8171dfaa3611f32622ca4658d53f31c79077
Successfully built gym


In [19]:
!pip install opencv-python

   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ------- -------------------------------- 6.8/38.8 MB 46.5 MB/s eta 0:00:01
   ------------------ --------------------- 18.1/38.8 MB 47.5 MB/s eta 0:00:01
   ------------------------------- -------- 30.4/38.8 MB 52.2 MB/s eta 0:00:01
   ---------------------------------------  38.8/38.8 MB 51.4 MB/s eta 0:00:01
   ---------------------------------------- 38.8/38.8 MB 45.7 MB/s eta 0:00:00


In [63]:
!pip install matplotlib

In [77]:
!pip install stable-baselines3[extra]

   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 1.4/1.4 MB 9.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/5.5 MB ? eta -:--:--
   --------------- ------------------------ 2.1/5.5 MB 9.8 MB/s eta 0:00:01
   -------------------------------- ------- 4.5/5.5 MB 11.2 MB/s eta 0:00:01
   -------------------------------------- - 5.2/5.5 MB 9.1 MB/s eta 0:00:01
   ---------------------------------------- 5.5/5.5 MB 7.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/4.4 MB ? eta -:--:--
   ----------------------- ---------------- 2.6/4.4 MB 15.1 MB/s eta 0:00:01
   ---------------------------------------- 4.4/4.4 MB 11.5 MB/s eta 0:00:00


In [81]:
!pip install torch

In [105]:
from vizdoom import *
import random
import time
import numpy as np
from gymnasium import Env
from gymnasium.spaces import Discrete, Box
import cv2
from matplotlib import pyplot as plt
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

In [89]:
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common import env_checker
from stable_baselines3 import PPO

In [91]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [93]:
class VizDoomGym(Env):
    def __init__(self, render=False):
        super().__init__()
        self.game = DoomGame()
        self.game.load_config('github/VizDoom/scenarios/basic.cfg')

        if render:
            self.game.set_window_visible(True)
        else:
            self.game.set_window_visible(False)
            
        self.game.init()

        #Sets up observation space(image of game) and action space
        self.observation_space = Box(low=0, high=255, shape=(1, 100, 160), dtype = np.uint8)
        self.action_space = Discrete(3)
        
    def step(self, action):
        actions = np.identity(3, dtype=np.uint8)
        reward = self.game.make_action(actions[action], 4)
        terminated = self.game.is_episode_finished()
        truncated = False
        info = {}
        
        if not terminated:
            state = self.game.get_state().screen_buffer
            state = self.preprocess(state)
        else:
            state = np.zeros(self.observation_space.shape, dtype=np.uint)
        
        return state, reward, terminated, truncated, info
        
    def render(self):
        pass
            
    def reset(self, seed =None, options = None):
        if seed is not None:
            self.game.set_seed(seed)

        self.game.new_episode()
        state = self.game.get_state().screen_buffer
        observation = self.preprocess(state)
        info = {}
        return observation, info
        
    #Grayscale game frame and resize it 
    def preprocess(self, observation):
        gray = cv2.cvtColor(np.moveaxis(observation, 0, -1), cv2.COLOR_BGR2GRAY)
        resized  = cv2.resize(gray, (160, 100), interpolation=cv2.INTER_AREA)
        state = np.reshape(resized, (1, 100, 160))
        return state.astype(np.float32) / 255.0
        
    def close(self):
        self.game.close()
    

In [132]:
def evaluate_policy(model, env, trials=5):
    total_rewards = []
    for episode in range(trials):
        state, _ = env.reset()
        state = torch.FloatTensor(state).unsqueeze(0)
        done = False
        total_reward = 0
        while not done:
            with torch.no_grad():
                logits, _ = agent.get_action(state)
                probs = torch.softmax(logits, dim=-1)
                action = torch.argmax(probs, dim=-1).item()
                next_state, reward, terminated, truncated, info = env.step(action)
                state = torch.FloatTensor(next_state).unsqueeze(0)
                total_reward += reward
                done = terminated
        total_rewards.append(total_reward)
        print(f"Episode {episode + 1} Reward: {total_reward}")
        average_reward = np.mean(total_rewards)
        print(f"Average Reward over {trials} episodes: {average_reward}")
        
                

In [97]:
class PPOAgent(nn.Module):
    def __init__(self, action_space):
        super(PPOAgent, self).__init__()
        self.conv = nn.Sequential(
            #based logic on stablebaselines
            nn.Conv2d(1, 32, kernel_size=8, stride=4),  # Output: [32, 24, 39]
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),  # Output: [64, 11, 18]
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),  # Output: [64, 9, 16]
            nn.ReLU(),
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 9 * 16, 512),
            nn.ReLU(),
        )
        self.policy_head = nn.Linear(512, action_space.n)
        self.value_head = nn.Linear(512, 1)
        
    def forward(self, x):
        x = self.conv(x)
        x = self.fc(x)
        return x
    
    def get_action(self, state):
        x = self.forward(state)
        logits = self.policy_head(x)
        value = self.value_head(x)
        return logits, value


In [160]:
from torch.utils.tensorboard import SummaryWriter
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

class PPO:
    def __init__(self, env, agent, run_name, epochs=10, clip_eps=0.2, gamma=0.99, lr=1e-4, batch_size=3000):
        self.env = env
        self.agent = agent
        self.optimizer = optim.Adam(self.agent.parameters(), lr=lr)
        self.epochs = epochs
        self.clip_eps = clip_eps
        self.gamma = gamma
        self.batch_size = batch_size

        
        log_dir = os.path.join("ppo_logs", run_name)
        self.writer = SummaryWriter(log_dir)
        self.global_step = 0  

    def compute_returns(self, rewards, dones, gamma):
        returns = []
        R = 0
        for step in reversed(range(len(rewards))):
            if dones[step]:
                R = 0  
            R = rewards[step] + gamma * R
            returns.insert(0, R)
        return returns

    def learn(self, total_timesteps):
        state, _ = self.env.reset()
        state = torch.FloatTensor(state).unsqueeze(0)
        timesteps = 0
        episode_rewards = []
        episode_reward = 0
        episode = 0

        while timesteps < total_timesteps:
            states = []
            actions = []
            log_probs = []
            rewards = []
            values = []
            dones = []

            for _ in range(self.batch_size):
                logits, value = self.agent.get_action(state)
                probs = torch.softmax(logits, dim=-1)
                dist = torch.distributions.Categorical(probs)
                action = dist.sample()
                log_prob = dist.log_prob(action)

                next_state, reward, terminated, truncated, _ = self.env.step(action.item())
                next_state = torch.FloatTensor(next_state).unsqueeze(0)

                states.append(state)
                actions.append(action)
                log_probs.append(log_prob.detach())
                rewards.append(reward)
                values.append(value.detach().item())
                dones.append(terminated)

                episode_reward += reward
                self.global_step += 1  

                if terminated:
                    episode_rewards.append(episode_reward)
                    print(f"Episode {episode} Reward: {episode_reward}")

                    # Calculate and log average reward
                    if len(episode_rewards) >= 10:
                        average_reward = np.mean(episode_rewards[-10:])
                    else:
                        average_reward = np.mean(episode_rewards)
                    self.writer.add_scalar("Average Reward", average_reward, self.global_step)

                    episode_reward = 0
                    episode += 1
                    state, _ = self.env.reset()
                    state = torch.FloatTensor(state).unsqueeze(0)
                else:
                    state = next_state

                timesteps += 1

            returns = self.compute_returns(rewards, dones, self.gamma)

            states = torch.cat(states)
            actions = torch.tensor(actions).unsqueeze(-1)
            log_probs = torch.stack(log_probs)
            returns = torch.tensor(returns).unsqueeze(-1)
            values = torch.tensor(values).unsqueeze(-1)

            advantages = returns - values
            advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)


            policy_losses = []
            value_losses = []
            entropies = []

            for _ in range(self.epochs):
                logits, value = self.agent.get_action(states)
                probs = torch.softmax(logits, dim=-1)
                dist = torch.distributions.Categorical(probs)
                new_log_probs = dist.log_prob(actions.squeeze(-1)).unsqueeze(-1)
                entropy = dist.entropy().mean()

                ratio = (new_log_probs - log_probs).exp()
                surr1 = ratio * advantages
                surr2 = torch.clamp(ratio, 1 - self.clip_eps, 1 + self.clip_eps) * advantages

                policy_loss = -torch.min(surr1, surr2).mean()
                value_loss = nn.SmoothL1Loss()(value, returns)
                loss = policy_loss + 0.5 * value_loss - 0.03 * entropy

                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()


                policy_losses.append(policy_loss.item())
                value_losses.append(value_loss.item())
                entropies.append(entropy.item())


            avg_policy_loss = np.mean(policy_losses)
            avg_value_loss = np.mean(value_losses)
            avg_entropy = np.mean(entropies)

            self.writer.add_scalar("Policy Loss", avg_policy_loss, self.global_step)
            self.writer.add_scalar("Value Loss", avg_value_loss, self.global_step)
            self.writer.add_scalar("Entropy", avg_entropy, self.global_step)


            print(f"Timesteps: {timesteps}, Policy Loss: {avg_policy_loss}, Value Loss: {avg_value_loss}, Entropy: {avg_entropy}")


        self.writer.close()

In [162]:
class PPOCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(PPOCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [150]:
basic_file_list = []

In [164]:
#Training the model
env = VizDoomGym(render=False)
agent = PPOAgent(env.action_space)
run_name= "basic_2"
ppo = PPO(
    env=env,
    agent=agent,
    run_name=run_name)
total_timesteps = 50000
ppo.learn(total_timesteps)

Episode 0 Reward: 0.0
Episode 1 Reward: -360.0
Episode 2 Reward: 91.0
Episode 3 Reward: -365.0
Episode 4 Reward: 75.0
Episode 5 Reward: 68.0
Episode 6 Reward: 51.0
Episode 7 Reward: -173.0
Episode 8 Reward: 75.0
Episode 9 Reward: 95.0
Episode 10 Reward: 87.0
Episode 11 Reward: 79.0
Episode 12 Reward: 87.0
Episode 13 Reward: -370.0
Episode 14 Reward: 87.0
Episode 15 Reward: 95.0
Episode 16 Reward: 87.0
Episode 17 Reward: -370.0
Episode 18 Reward: 68.0
Episode 19 Reward: 87.0
Episode 20 Reward: -380.0
Episode 21 Reward: 95.0
Episode 22 Reward: 95.0
Episode 23 Reward: 35.0
Episode 24 Reward: 87.0
Episode 25 Reward: 13.0
Episode 26 Reward: 95.0
Episode 27 Reward: 95.0
Episode 28 Reward: -370.0
Episode 29 Reward: -375.0
Episode 30 Reward: -12.0
Episode 31 Reward: 42.0
Episode 32 Reward: 33.0
Episode 33 Reward: -50.0
Episode 34 Reward: -355.0
Episode 35 Reward: 79.0
Episode 36 Reward: 37.0
Episode 37 Reward: -365.0
Episode 38 Reward: 29.0
Episode 39 Reward: -84.0
Episode 40 Reward: -77.0
Epi

In [152]:
filename = f'ppo_vizdoom_agent_{run_name}.pth'
torch.save(agent.state_dict(), filename)
basic_file_list.append(filename)

In [158]:
evaluate_policy(agent, env, trials=5)

Episode 1 Reward: 95.0
Average Reward over 5 episodes: 95.0
Episode 2 Reward: 95.0
Average Reward over 5 episodes: 95.0
Episode 3 Reward: 95.0
Average Reward over 5 episodes: 95.0
Episode 4 Reward: 95.0
Average Reward over 5 episodes: 95.0
Episode 5 Reward: 95.0
Average Reward over 5 episodes: 95.0


In [ ]:
CHECKPOINT_DIR = './train/train_basic'
LOG_DIR = './logs/log_basic'

In [21]:
callback = PPOCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

In [23]:
env = VizDoomGym()
model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=1, learning_rate=0.0001, n_steps=2048)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


In [11]:
model.learn(total_timesteps = 100000, callback = callback)

NameError: name 'model' is not defined

In [141]:
env_checker.check_env(env)

In [41]:
env.close()